In [0]:
import os
import sys
import math
import time

import tensorflow as tf
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

# Load CIFAR10 dataset

![](https://appliedmachinelearning.files.wordpress.com/2018/03/cifar2.jpg)

In [0]:
cifar = keras.datasets.cifar10

(X_train, y_train), (X_test, y_test) = cifar.load_data()

X_train = X_train / 255
X_test = X_test / 255

enc = OneHotEncoder(sparse=False, categories='auto')

y_train = enc.fit_transform(y_train.reshape(-1, 1))
y_test = enc.transform(y_test.reshape(-1, 1))

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# Create Convolutional Neural Networks for CIFAR images classification


## What convolutional neural network is?

![](https://cdn-images-1.medium.com/max/1400/1*vkQ0hXDaQv57sALXAJquxA.jpeg)

## What is the image filter

![](https://www.researchgate.net/profile/Seiichi_Uchida/publication/236125496/figure/fig5/AS:214053629763589@1428045770840/Effect-of-three-different-edge-detection-filters-Laplacian-Canny-and-Sobel-filters.png)

## What is the convolution operation?

![](https://cdn-images-1.medium.com/max/800/1*Zx-ZMLKab7VOCQTxdZ1OAw.gif)

![](https://cdn-images-1.medium.com/max/800/1*ciDgQEjViWLnCbmX-EeSrA.gif)

## What is the pooling operation

![](https://cdn-images-1.medium.com/max/600/1*uoWYsCV5vBU8SHFPAPao-w.gif)

## Define the network

In the following section we will use the tensorflow lower level API to define the convolutional neural network that will classify CIFAR10 images and all auxillary functions that will be useful during training.

**Create placeholders for training data.**

Remember about a propper shape of training images (every training example is an image shaped 32x32x3 pixels) and labels (In training dataset labels are in one-hot-encoding form).

In [0]:
x_placeholder = tf.placeholder(tf.float32, [None, 32, 32, 3])
y_placeholder = tf.placeholder(tf.float32, [None, 10])

** Create ConvNet **

Define the following layers of your new model:

1.   Convolutional layer [keras.layers.Conv2d](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) that will process the input data (feed by placeholder). The layer should have 32 filters, with kernel size equal to 3, same padding and ReLU activation function.
2.   Another convolutional layer with 32 filters, kernel size equal to 3, same padding and ReLU activation function.
3.   Max pooling layer [keras.layers.MaxPool2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D), with pool size equal to 2.
4.   Convolutional layer with 64 filters, kernel size equal to 3, same padding and ReLU activation function.
5.   Another convolutional layer with 64 filters, kernel size equal to 3, same padding and ReLU activation function.
6.   Max pooling layer with pool size equal to 2.
7.   Layer that will flatten the result of convolutions [keras.layers.Flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten).
8.   Dense layer ([keras.layers.Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)) with 128 neurons (aka units), and the "relu" activation function.
4.  A final Dense layer with 10 neurons (one per class), , without any activation function -- returning the logits (not softmax), together with the specified loss function ensures the numerical stability.

In [0]:
conv1 = keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu")(x_placeholder)
conv2 = keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu")(conv1)
pool1 = keras.layers.MaxPool2D(pool_size=2)(conv2)
conv3 = keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu")(pool1)
conv4 = keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu")(conv3)
pool2 = keras.layers.MaxPool2D(pool_size=2)(conv4)
flatten = keras.layers.Flatten()(pool2)
dense = keras.layers.Dense(128, activation="relu")(flatten)
logits = keras.layers.Dense(10, activation=None)(dense)

**Define the loss function**

This is the place to define the loss function for our model. Cross-entropy is the classical approach to use in the multi-label classification task.  However for numerical stability we didn't use the softmax activation function (so as we are taking the log of softmax -- logits).

This time, instead of using [tf.nn.softmax_cross_entropy_with_logits_v2](https://www.tensorflow.org/api_docs/python/tf/nn/softmax_cross_entropy_with_logits_v2), you could use the [softmax_cross_entropy](https://www.tensorflow.org/api_docs/python/tf/losses/softmax_cross_entropy) function from [tf.losses](https://www.tensorflow.org/api_docs/python/tf/losses) module, which is the module with some predefined loss functions.

In [0]:
cross_entropy = tf.losses.softmax_cross_entropy(onehot_labels=y_placeholder, logits=logits)

**Define the optimizer**

Use the [Adam optimizer](https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer) and set it to minimize the loss function.

In [0]:
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

** Define operation for accuracy calculations **

This time instead od writing it by ourselves, we will use the [accuracy](https://www.tensorflow.org/api_docs/python/tf/metrics/accuracy) function from [tf.metrics](https://www.tensorflow.org/api_docs/python/tf/metrics) module, which is the module with some predefined metrics for validation of neural networks.

We cannot test our network on the whole test set, as it won't fit in the memory of our GPU. Instead we could calculate our metrics in batches. In order to do that, remember about *update_op* returned by accuracy function, [like in the following link](http://ronny.rest/blog/post_2017_09_11_tf_metrics/).

In [0]:
accuracy, update_op = tf.metrics.accuracy(labels=tf.argmax(y_placeholder, 1), predictions=tf.argmax(logits, 1), name="accuracy")

In [0]:
# Isolate the variables stored behind the scenes by the metric operation
running_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="accuracy")

# Define initializer to initialize/reset running variables
running_vars_initializer = tf.variables_initializer(var_list=running_vars)

** Define the function that calculates the accuracy in batches **

In [0]:
def calculate_accuracy_batches(X_test, y_test, batch_size, accuracy, update_op, sess):    
    for i in range(0, X_test.shape[0], batch_size):
        y_batch = y_test[i:(i + batch_size), :]
        x_batch = X_test[i:(i + batch_size), :]
        sess.run(update_op, feed_dict={
            x_placeholder: x_batch, y_placeholder: y_batch})
        
    return sess.run(accuracy)

## Train the network

In [0]:
epoch_num = 10
batch_size = 128
set_size = X_train.shape[0]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(running_vars_initializer)
    
    validation_accuracy = calculate_accuracy_batches(X_test, y_test, batch_size,
                                                     accuracy, update_op, sess)
    print('Validation accuracy before training: {}'.format(validation_accuracy))
    
    for epoch in range(epoch_num):
        start_time = time.time()
        
        perm = np.random.permutation(set_size)
        X_train = X_train[perm, :]
        y_train = y_train[perm, :]

        for i in range(0, set_size, batch_size):
            step_size = min(batch_size, set_size - i)

            if step_size > 1:
                y_batch = y_train[i:(i + step_size), :]
                x_batch = X_train[i:(i + step_size), :]
                train_step.run(feed_dict={
                                x_placeholder: x_batch, y_placeholder: y_batch})
                
        validation_accuracy = calculate_accuracy_batches(X_test, y_test, 
                                          batch_size, accuracy, update_op, sess)
        end_time = time.time()
        print('step: {}, validation accuracy: {}, epoch time: {}'.format(epoch, validation_accuracy, end_time-start_time))


    # Print the test set accuracy
    test_accuracy = calculate_accuracy_batches(X_test, y_test, batch_size, accuracy, update_op, sess)
    print('test accuracy: {}'.format(test_accuracy))

# TensorFlow Datasets

GPUs and TPUs can radically reduce the time required to execute a single training step. Achieving peak performance requires an efficient input pipeline that delivers data for the next step before the current step has finished. The tf.data API helps to build flexible and efficient input pipelines.


\\

A typical TensorFlow training input pipeline can be framed as an ETL process:

1.    Extract: Read data from persistent storage -- either local (e.g. HDD or SSD) or remote (e.g. GCS or HDFS).
2.    Transform: Use CPU cores to parse and perform preprocessing operations on the data such as image decompression, data augmentation transformations (such as random crop, flips, and color distortions), shuffling, and batching.
3.    Load: Load the transformed data onto the accelerator device(s) (for example, GPU(s) or TPU(s)) that execute the machine learning model.

This pattern effectively utilizes the CPU, while reserving the accelerator for the heavy lifting of training your model. In addition, viewing input pipelines as an ETL process provides structure that facilitates the application of performance optimizations.

**In a naive *feed_dict* pipeline the GPU always sits by idly whenever it has to wait for the CPU to provide it with the next batch of data.**

![](https://dominikschmidt.xyz/tensorflow-data-pipeline/assets/feed_dict_pipeline.png)

**A *tf.data* pipeline, however, can prefetch the next batches asynchronously to minimize the total idle time. You can further speed up the pipeline by parallelizing the loading and preprocessing operations.**
![](https://dominikschmidt.xyz/tensorflow-data-pipeline/assets/tf_data_pipeline.png)

## Code examples

More examples on [datasets guide](https://www.tensorflow.org/guide/datasets)

**Dataset with one-shot iterator**

A one-shot iterator is the simplest form of iterator, which only supports iterating once through a dataset, with no need for explicit initialization.

In [0]:
with tf.Session() as sess:
    dataset = tf.data.Dataset.range(100)
    iterator = dataset.make_one_shot_iterator()
    next_element = iterator.get_next()

    for i in range(100):
        value = sess.run(next_element)
        assert i == value

** Dataset with initializable iterator **

An initializable iterator requires you to run an explicit iterator.initializer operation before using it. In exchange for this inconvenience, it enables you to parameterize the definition of the dataset, using one or more tf.placeholder() tensors that can be fed when you initialize the iterator.

In [0]:
with tf.Session() as sess:
    max_value = tf.placeholder(tf.int64, shape=[])
    dataset = tf.data.Dataset.range(max_value)
    iterator = dataset.make_initializable_iterator()
    next_element = iterator.get_next()

    # Initialize an iterator over a dataset with 10 elements.
    sess.run(iterator.initializer, feed_dict={max_value: 10})
    for i in range(10):
        value = sess.run(next_element)
        assert i == value

    # Initialize the same iterator over a dataset with 100 elements.
    sess.run(iterator.initializer, feed_dict={max_value: 100})
    for i in range(100):
        value = sess.run(next_element)
        assert i == value

** End of dataset **

If the iterator reaches the end of the dataset, executing the Iterator.get_next() operation will raise a tf.errors.OutOfRangeError. After this point the iterator will be in an unusable state, and you must initialize it again if you want to use it further.

In [0]:
with tf.Session() as sess:
    dataset = tf.data.Dataset.range(5)
    iterator = dataset.make_initializable_iterator()
    next_element = iterator.get_next()

    # Typically `result` will be the output of a model, or an optimizer's
    # training operation.
    result = tf.add(next_element, next_element)

    sess.run(iterator.initializer)
    print(sess.run(result))  # ==> "0"
    print(sess.run(result))  # ==> "2"
    print(sess.run(result))  # ==> "4"
    print(sess.run(result))  # ==> "6"
    print(sess.run(result))  # ==> "8"
    
    try:
        sess.run(result)
    except tf.errors.OutOfRangeError:
        print("End of dataset")  # ==> "End of dataset"

**Preprocessing data with Dataset.map()**

The Dataset.map(f) transformation produces a new dataset by applying a given function f to each element of the input dataset. It is based on the map() function that is commonly applied to lists (and other structures) in functional programming languages. The function f takes the tf.Tensor objects that represent a single element in the input, and returns the tf.Tensor objects that will represent a single element in the new dataset. Its implementation uses standard TensorFlow operations to transform one element into another.

In [0]:
def parse_function(value):
    return value * value

with tf.Session() as sess:
    dataset = tf.data.Dataset.range(10)
    dataset = dataset.map(map_func=parse_function, num_parallel_calls=2)
    iterator = dataset.make_one_shot_iterator()
    next_element = iterator.get_next()        
        
    try:
        while True:
            value = sess.run(next_element)
            print(value)
    except tf.errors.OutOfRangeError:
        print("End of dataset")

# Use Dataset for ConvNet training


In the following section, we will replace manual batch preparation by the [tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) pipeline.

Our model is already defined, so that we will only prepare the training pipeline.

In [0]:
epoch_num = 10
batch_size = 128

If all of your input data fit in memory, the simplest way to create a Dataset from them is to convert them to tf.Tensor objects and use **Dataset.from_tensor_slices()**.

However doing so will embed the features and labels arrays in your TensorFlow graph as tf.constant() operations. This works well for a small dataset, but wastes memory---because the contents of the array will be copied multiple times---and can run into the 2GB limit for the tf.GraphDef protocol buffer.

As an alternative, you can define the Dataset in terms of **tf.placeholder()** tensors, [like in the following link](https://www.tensorflow.org/guide/datasets#consuming_numpy_arrays), and feed the NumPy arrays when you initialize an Iterator over the dataset.

## Define the tf dataset

**Create placeholders for dataset features and labels.**

Remember about a propper shape and type. One placeholder will be feeded with images and another with one-hot encodings.

In [0]:
features_placeholder = tf.placeholder(x_placeholder.dtype, x_placeholder.shape)
labels_placeholder = tf.placeholder(y_placeholder.dtype, y_placeholder.shape)

** Define the dataset from placeholders **

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))

** Define the dataset structure **

We will use the dataset to train our model, so that we want to **shuffle** our data before every epoch, we also want to train our model in **batches**.  Setting [prefetch](https://www.tensorflow.org/guide/performance/datasets#pipelining) buffer could be also useful to spped up our training.

In [0]:
dataset = dataset.shuffle(buffer_size=50000)
dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(buffer_size=5)

** Create an iterator **

Create the [initializable iterator](https://www.tensorflow.org/guide/datasets#creating_an_iterator) and get its symbolic output.

In [0]:
iterator = dataset.make_initializable_iterator()
next_example, next_label = iterator.get_next()

**Define the function that calculates the accuracy in batches and uses the created dataset**

In [0]:
def calculate_accuracy_batches_dataset(X_test, y_test, iterator, accuracy, update_op, sess):
    sess.run(iterator.initializer, feed_dict={features_placeholder: X_test,
                                              labels_placeholder: y_test})
    
    try:
        while True:
            x_batch, y_batch = sess.run([next_example, next_label])
            sess.run(update_op, feed_dict={x_placeholder: x_batch, y_placeholder: y_batch})
    except tf.errors.OutOfRangeError:
        pass
        
    return sess.run(accuracy)

## Train the network

In [0]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(running_vars_initializer)
    
    validation_accuracy = calculate_accuracy_batches_dataset(X_test, y_test, iterator, accuracy, update_op, sess)
    print('Validation accuracy before training: {}'.format(validation_accuracy))
    
    for epoch in range(epoch_num):
        start_time = time.time()
        sess.run(iterator.initializer, feed_dict={features_placeholder: X_train,
                                                  labels_placeholder: y_train})
        
        try:
            while True:
                x_batch, y_batch = sess.run([next_example, next_label])
                train_step.run(feed_dict={x_placeholder: x_batch, y_placeholder: y_batch})
        except tf.errors.OutOfRangeError:
            pass
        
        validation_accuracy = calculate_accuracy_batches_dataset(X_test, y_test, iterator, accuracy, update_op, sess)
        end_time = time.time()
        print('step: {}, validation accuracy: {}, epoch time: {}'.format(epoch, validation_accuracy, end_time-start_time))
        
    # Print the test set accuracy
    test_accuracy = calculate_accuracy_batches_dataset(X_test, y_test, iterator, accuracy, update_op, sess)
    print('test accuracy: {}'.format(test_accuracy))

**Note: You can also use the tf dataset to train model in keras sequential or keras functional API [link](https://stackoverflow.com/questions/52736517/tensorflow-keras-with-tf-dataset-input)**

# Images augmentation

Data augmentation can artificially increase the size of our training datasets and therefore increase the metrics of our neural networks.

\\

In TensorFlow images augmentation could be computed on CPU and on GPU. Images augmentation operations are defined in [tf.image](https://www.tensorflow.org/api_docs/python/tf/image) module.

## Some images augmentation examples

** Original image**
![](https://cdn-images-1.medium.com/max/1000/1*EqjJ1tEsRoc4FNptI9teeA.jpeg)

**Flipped images**

images flipping could be done by using [tf.image.random_flip_left_right](https://www.tensorflow.org/api_docs/python/tf/image/random_flip_left_right) and [tf.image.random_flip_up_down](https://www.tensorflow.org/api_docs/python/tf/image/random_flip_up_down) functions

![](https://cdn-images-1.medium.com/max/800/1*UYDREwlBU0ua9-g5ccRWgQ.png)

![](https://cdn-images-1.medium.com/max/800/1*NmIPTEmki86rAlDkqOPHiQ.png)

** Images rotation **

Which could be done by using [tf.image.rot90](https://www.tensorflow.org/api_docs/python/tf/image/rot90) and [tf.contrib.image.rotate](https://www.tensorflow.org/api_docs/python/tf/contrib/image/rotate) functions. 

**Note:** Warning, tf.contrib will be deleted in tf 2.0

![](https://cdn-images-1.medium.com/max/800/1*xtZM7snC-6Nkq4_iB1RnKg.png)

# ConvNet training with images augmentation

In the following section we will define some image augmentations that will be proceeded by our dataset on CPU, in the meantime when batch gradient will be computed on the GPU.

## Define the augmentation and tf dataset

**Define the augmentation function**

Define the augmentation function with using of [tf.image](https://www.tensorflow.org/api_docs/python/tf/image) module. You could use any augmentation that you like.

Please notice that since the pipeline parsing function should be running on CPU, we have to specify the [device](https://www.tensorflow.org/api_docs/python/tf/device) manually.

In [0]:
def cpu_augmentation_fn(image, label):
    with tf.device('/cpu:0'):
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        image = tf.cond(
            tf.less(tf.random_uniform(()), 0.5),
            lambda: tf.contrib.image.rotate(
                image,
                tf.random_uniform((), -30 / 180 * math.pi, 30 / 180 * math.pi)
            ),
            lambda: tf.identity(image)
        )

    return image, label

** Define the dataset **

We will use the dataset to train our model, so that we want to **shuffle** our data before every epoch, we also want to train our model in **batches**.  Setting [prefetch](https://www.tensorflow.org/guide/performance/datasets#pipelining) buffer could be also useful to spped up our training. We also want to use our predefined augmentation as the dataset [parsing function](https://www.tensorflow.org/guide/datasets#preprocessing_data_with_datasetmap).

**Note:** Please notice that we don't want to make an augmentation during the testing of our network. Not shuffling during test time is also a good idea.

To handle this problem, we will define the function that creates the dataset with a given specified mode: *train/test*. And then, by calling this function two times, we will define the train and test datasets.

In [0]:
def create_dataset(mode='train', parse_fn=cpu_augmentation_fn, batch_size=128, buffer_size=50000, channels=3):
    features_placeholder = tf.placeholder(x_placeholder.dtype, x_placeholder.shape)
    labels_placeholder = tf.placeholder(y_placeholder.dtype, y_placeholder.shape)
    
    dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))
    
    if mode == 'train':
        dataset = dataset.map(map_func=parse_fn, num_parallel_calls=8)
        dataset = dataset.shuffle(buffer_size=buffer_size)
        
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=5)
    
    return dataset, features_placeholder, labels_placeholder

In [0]:
train_dataset, train_features_placeholder, train_labels_placeholder = create_dataset(mode='train')
val_dataset, val_features_placeholder, val_labels_placeholder = create_dataset(mode='valid')

** Create an iterator **

Create the [initializable iterator](https://www.tensorflow.org/guide/datasets#creating_an_iterator) and get its symbolic output for both training and testing datasets.

In [0]:
train_iterator = train_dataset.make_initializable_iterator()
next_train_example, next_train_label = train_iterator.get_next()

In [0]:
val_iterator = val_dataset.make_initializable_iterator()
next_val_example, next_val_label = val_iterator.get_next()

**Define the function that calculates the accuracy in batches and uses the created test dataset**

In [0]:
def calculate_accuracy_batches_dataset(X_test, y_test, accuracy, update_op, sess):
    sess.run(val_iterator.initializer, feed_dict={val_features_placeholder: X_test,
                                                  val_labels_placeholder: y_test})
    
    try:
        while True:
            x_batch, y_batch = sess.run([next_val_example, next_val_label])
            sess.run(update_op, feed_dict={x_placeholder: x_batch, y_placeholder: y_batch})
    except tf.errors.OutOfRangeError:
        pass
        
    return sess.run(accuracy)

## Train the network

In [0]:
epoch_num = 10

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(running_vars_initializer)
    
    validation_accuracy = calculate_accuracy_batches_dataset(X_test, y_test, accuracy, update_op, sess)
    print('Validation accuracy before training: {}'.format(validation_accuracy))
    
    for epoch in range(epoch_num):
        start_time = time.time()
        
        sess.run(train_iterator.initializer, feed_dict={train_features_placeholder: X_train,
                                                        train_labels_placeholder: y_train})
        
        try:
            while True:
                x_batch, y_batch = sess.run([next_train_example, next_train_label])
                train_step.run(feed_dict={x_placeholder: x_batch, y_placeholder: y_batch})
        except tf.errors.OutOfRangeError:
            pass
        
        validation_accuracy = calculate_accuracy_batches_dataset(X_test, y_test, accuracy, update_op, sess)
        end_time = time.time()
        print('step: {}, validation accuracy: {}, epoch time: {}'.format(epoch, validation_accuracy, end_time-start_time))
        
    # Print the test set accuracy
    test_accuracy = calculate_accuracy_batches_dataset(X_test, y_test, accuracy, update_op, sess)
    print('test accuracy: {}'.format(test_accuracy))

## Compare with training without dataset pipeline

Compare the epoch times of training with augmentation with using tf dataset and without using tf dataset

In [0]:
def cpu_augmentation_fn_v2(image):
    with tf.device('/cpu:0'):
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        image = tf.cond(
            tf.less(tf.random_uniform(()), 0.5),
            lambda: tf.contrib.image.rotate(
                image,
                tf.random_uniform((), -30 / 180 * math.pi, 30 / 180 * math.pi)
            ),
            lambda: tf.identity(image)
        )

    return image

In [0]:
### 1746 seconds per epoch !!!
epoch_num = 1
batch_size = 128
set_size = X_train.shape[0]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(running_vars_initializer)
    
    validation_accuracy = calculate_accuracy_batches(X_test, y_test, batch_size,
                                                     accuracy, update_op, sess)
    print('Validation accuracy before training: {}'.format(validation_accuracy))
    
    for epoch in range(epoch_num):
        start_time = time.time()
        
        perm = np.random.permutation(set_size)
        X_train = X_train[perm, :]
        y_train = y_train[perm, :]

        for i in range(0, set_size, batch_size):
            step_size = min(batch_size, set_size - i)

            if step_size > 1:
                y_batch = y_train[i:(i + step_size), :]
                x_batch = X_train[i:(i + step_size), :]
                x_batch = tf.map_fn(cpu_augmentation_fn_v2, x_batch)
                train_step.run(feed_dict={
                                x_placeholder: x_batch.eval(), y_placeholder: y_batch})
                
        validation_accuracy = calculate_accuracy_batches(X_test, y_test, 
                                          batch_size, accuracy, update_op, sess)
        end_time = time.time()
        print('step: {}, validation accuracy: {}, epoch time: {}'.format(epoch, validation_accuracy, end_time-start_time))


    # Print the test set accuracy
    test_accuracy = calculate_accuracy_batches(X_test, y_test, batch_size, accuracy, update_op, sess)
    print('test accuracy: {}'.format(test_accuracy))

# Images sources

Images used in this notebook comes from the following web pages:
1.   https://appliedmachinelearning.blog/2018/03/24/achieving-90-accuracy-in-object-recognition-task-on-cifar-10-dataset-with-keras-convolutional-neural-networks/
2.   https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53
3.   https://towardsdatascience.com/intuitively-understanding-convolutions-for-deep-learning-1f6f42faee1
4.   https://dominikschmidt.xyz/tensorflow-data-pipeline/
5.   https://www.researchgate.net/figure/Effect-of-three-different-edge-detection-filters-Laplacian-Canny-and-Sobel-filters_fig5_236125496


# Other references

Text is also inspired by:
1.   [TensorFlow guide](https://www.tensorflow.org/guide)
2.   [DL with TF](https://github.com/ageron/tf2_course) course by Aurelien Geron


